In [6]:

import pandas as pd
import numpy as np

non_value_columns = np.array([
    {"name": 'DT_ANO_LETIVO_INICIO'},
    {"name": 'DT_ANO_LETIVO_TERMINO'},
    {"name": 'NO_ENTIDADE'},
    {"name": 'NO_MUNICIPIO'},
    {"name": 'NO_UF'},
    {"name": 'SG_UF'},
    {"name": 'NO_REGIAO'},
    {"name": 'NU_ANO_CENSO'},   
    { "name":"TP_CATEGORIA_ESCOLA_PRIVADA"},
    {"name":"IN_COZINHA"},
    {"name":"IN_ESGOTO_INEXISTENTE"},
    {"name":"IN_ENERGIA_INEXISTENTE"},
    {"name":"IN_AREA_VERDE"},
    {"name":"IN_BANHEIRO"},
    {"name":"IN_DESPENSA"},
    {"name":"IN_SECRETARIA"},
    {"name":"IN_TRATAMENTO_LIXO_INEXISTENTE"},
    {"name":"IN_REFEITORIO"},
    {"name":"IN_BIBLIOTECA"}
])
columns = [x['name'] for x in non_value_columns]




df_school_census_final = pd.read_csv("data/microdados_ed_basica_2022/dados/microdados_ed_basica_2022.csv",encoding="ISO-8859-1", sep = ';')
df_school_census_final = df_school_census_final[columns]

for year in range(2012,2022):
    csv_file = f'data/microdados_ed_basica_{year}/dados/microdados_ed_basica_{year}.csv'
    df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')
    df_current = df_current[columns]
    df_school_census_final = pd.concat([df_school_census_final,df_current])
    
#df_school_census_final.to_csv(r'C:\Users\Matheus Rafalski\Downloads\export_dataframe.csv', index=False)


#df_school_census_columns = pd.read_csv("Downloads/export_dataframe.csv",encoding="utf8", sep = ',')
df_school_census_columns = df_school_census_final

columns = np.array([
    {"name": 'IN_COZINHA', "1": 80, "0":0},
    {"name": 'IN_BIBLIOTECA', "1": 100, "0":0},
    {"name": 'IN_ESGOTO_INEXISTENTE', "1": 0, "0":100},
    {"name": 'IN_ENERGIA_INEXISTENTE', "1": 0, "0":100},
    {"name": 'IN_AREA_VERDE', "1": 80, "0":0},
    {"name": 'IN_BANHEIRO', "1": 100, "0":0},
    {"name": 'IN_DESPENSA', "1": 80, "0":0},
    {"name": 'IN_SECRETARIA', "1": 100, "0":0},
    {"name": 'IN_TRATAMENTO_LIXO_INEXISTENTE', "1": 0, "0":80},
    {"name": 'IN_REFEITORIO', "1": 80, "0":0},
])

result = np.concatenate((columns, non_value_columns))


df_school_census_columns.dropna(axis=0, subset=['DT_ANO_LETIVO_INICIO'], inplace=True)
df_school_census_columns = df_school_census_columns.replace('',np.nan)
df_school_census_columns = df_school_census_columns[map(lambda column: column['name'], non_value_columns)]
df_school_census_columns = df_school_census_columns[df_school_census_columns["DT_ANO_LETIVO_INICIO"] != "0"]


df_school_census_columns = df_school_census_columns.fillna(value=df_school_census_columns.mode().iloc[0])
# Iterando sobre as colunas do dataframe
for column in columns:
    col = column['name']
    df_school_census_columns[col] = df_school_census_columns[col].astype(int)
    


def transform_categoria_escola(x):
    if x == 1:
        return "Particular"
    elif x == 2:
        return "Comunitária"
    elif x == 3:
        return "Confessional"
    elif x == 4:
        return "Filantrópica"
    else:
        return "Pública"

df_school_census_columns["TP_CATEGORIA_ESCOLA_PRIVADA"] = df_school_census_columns["TP_CATEGORIA_ESCOLA_PRIVADA"].apply(transform_categoria_escola)


def transform(x,column):
    x_object = column
    
    if x == 1:
        return  x_object['1']
    elif x == 0:
        return  x_object['0']
    else:
        return x

for i in columns:
    df_school_census_columns[i['name']] = df_school_census_columns[i['name']].apply(transform, column=i)

print("Dataframe resultante:")
columns_name= [x['name'] for x in columns]

estrutural_columns = df_school_census_columns[map(lambda column: column, columns_name)]
estrutural_columns = estrutural_columns.assign(total=estrutural_columns.sum(axis=1))
df_school_census_columns['total'] = estrutural_columns['total']
df_school_census_columns.drop(columns_name, axis=1, inplace=True)
print(df_school_census_columns.columns)
df_school_census_columns.drop(['DT_ANO_LETIVO_INICIO','DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)
print(df_school_census_columns.columns)
# merge pib csv in df_school_census and then removce unused columns and apply etl

def create_structure(x):
    if x >= 800:
        return "Ótima"
    elif (x >= 600) & (x < 800):
         return "Boa"
    elif (x >= 400) & (x < 600):
        return "Média"
    elif (x >= 200) & (x < 400):
        return "Ruim"
    elif (x >= 0) & (x < 200):
        return "Precária"
    
df_school_census_columns["Estrutura"] = df_school_census_columns["total"].apply(create_structure)
df_school_census_columns.drop(['total'], axis=1, inplace=True)
df_pib_cities = pd.read_excel("./data/pib_cities.xlsx")
df_pib_cities = df_pib_cities.drop(df_pib_cities.columns[[1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43]], axis=1)
df_pib_cities.columns.values[1] = "Município"
df_pib_cities.columns.values[2] = "PIB Per Capta"

df_school_census_columns.columns.values[0] = "Escola"
df_school_census_columns.columns.values[1] = "Município"
df_school_census_columns.columns.values[2] = "Estado"
df_school_census_columns.columns.values[3] = "Sigla"
df_school_census_columns.columns.values[4] = "Região"
df_school_census_columns.columns.values[5] = "Ano"
df_school_census_columns.columns.values[6] = "Categoria"
df_final = pd.merge(df_school_census_columns,df_pib_cities,how='inner',left_on=['Município', 'Ano'], right_on=['Município', 'Ano'])
df_final



#df_school_census_columns = df_school_census_columns.dropna(axis=0, subset=['DT_ANO_LETIVO_INICIO'])

C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_24064\3410553179.py:32: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_school_census_final = pd.read_csv("data/microdados_ed_basica_2022/dados/microdados_ed_basica_2022.csv",encoding="ISO-8859-1", sep = ';')
C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_24064\3410553179.py:37: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')
C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_24064\3410553179.py:37: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')
C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_24064\3410553179.py:37: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False

Dataframe resultante:
Index(['DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'NO_ENTIDADE',
       'NO_MUNICIPIO', 'NO_UF', 'SG_UF', 'NO_REGIAO', 'NU_ANO_CENSO',
       'TP_CATEGORIA_ESCOLA_PRIVADA', 'total'],
      dtype='object')
Index(['NO_ENTIDADE', 'NO_MUNICIPIO', 'NO_UF', 'SG_UF', 'NO_REGIAO',
       'NU_ANO_CENSO', 'TP_CATEGORIA_ESCOLA_PRIVADA', 'total'],
      dtype='object')
Index(['Ano', 'Nome do Município',
       'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)'],
      dtype='object')


,Escola,Município,Estado,Sigla,Região,Ano,Categoria,Estrutura,PIB Per Capta
0,EIEEF HAP BITT TUPARI,Alta Floresta D'Oeste,Rondônia,RO,Norte,2012,Pública,Ruim,13670.26
1,CEEJA LUIZ VAZ DE CAMOES,Alta Floresta D'Oeste,Rondônia,RO,Norte,2012,Pública,Média,13670.26
2,EMMEF 07 DE SETEMBRO,Alta Floresta D'Oeste,Rondônia,RO,Norte,2012,Pública,Ruim,13670.26
3,EMMEF AFONSO AURELIO PORTO,Alta Floresta D'Oeste,Rondônia,RO,Norte,2012,Pública,Ruim,13670.26
4,EMEIEF ANA NERY,Alta Floresta D'Oeste,Rondônia,RO,Norte,2012,Pública,Média,13670.26
...,...,...,...,...,...,...,...,...,...
1872627,ESC PEQUENO ENCANTO,Brasília,Distrito Federal,DF,Centro-Oeste,2020,Particular,Média,87016.16
1872628,COL ANCHIETA,Brasília,Distrito Federal,DF,Centro-Oeste,2020,Particular,Ótima,87016.16
1872629,COL MAPA,Brasília,Distrito Federal,DF,Centro-Oeste,2020,Particular,Média,87016.16
1872630,COBIAN - COL BIANGULO,Brasília,Distrito Federal,DF,Centro-Oeste,2020,Particular,Ótima,87016.16
